# 1. Check for duplicate reactions

## 1A. Load the dataset

In [1]:
from pathlib import Path
import os
import pandas as pd
os.chdir ('/data/karthiksankar2/SAR/Datasets/Coley_WLN/CompoundDiversification_GitHubCopy/2.ProcessedDataset')
dataset = pd.read_csv('Complete_dataset_results.csv', index_col = 0)

In [2]:
dataset.head()

,Reaction SMILES,reactionID,react_smiles,rxn_smiles,template,Template Verification (True/False)
0,[CH2:15]([CH:16]([CH3:17])[CH3:18])[Mg+:19].[C...,0,CON(C)C(=O)c1ccc(O)nc1,CON(C)[C:6]([c:5]1[cH:4][n:3][c:2]([OH:1])[cH:...,[#7;a:5]:[c:4]:[c:3]-[C;H0;D3;+0:1](=[O;D1;H0:...,True
1,[CH3:14][NH2:15].[N+:1](=[O:2])([O-:3])[c:4]1[...,1,O=C(O)c1ccc(Cl)c([N+](=O)[O-])c1,Cl[c:12]1[c:4]([N+:1](=[O:2])[O-:3])[cH:5][c:6...,[Cl;H0;D1;+0]-[c;H0;D3;+0:1](:[c:2]):[c:3]>>C-...,True
2,[CH2:1]([CH3:2])[n:3]1[cH:4][c:5]([C:22](=[O:2...,2,CCn1cc(C(=O)O)c(=O)c2cc(F)c(-c3ccc(N)cc3)cc21,[CH2:1]([CH3:2])[n:3]1[cH:4][c:5]([C:22](=[O:2...,[NH2;D1;+0:1]-[c:2]>>O=C-[NH;D2;+0:1]-[c:2],True
3,[Cl:1][C:2]([N:3]([CH3:4])[CH3:5])=[C:6]([CH3:...,3,COCC(C)Oc1cc(Oc2cnc(C(=O)N3CCC3)cn2)cc(C(=O)O)c1,O=[C:25]([c:24]1[cH:23][c:22]([O:21][c:18]2[cH...,[O;H0;D1;+0]=[C;H0;D3;+0:1](-[OH;D1;+0:2])-[c:...,True
4,[Cl:11][c:12]1[c:13]2[c:14]([n:15][c:16](-[c:1...,4,Clc1cc2c(Cl)nc(-c3ccncc3)nc2s1,Cl[c:12]1[c:13]2[c:14]([n:15][c:16](-[c:18]3[c...,[#16;a:6]:[c:5]1:[#7;a:4]:[c:3]:[#7;a:2]:[c;H0...,True


In [3]:
os.chdir ('/data/karthiksankar2/SAR/Datasets/Coley_WLN/CompoundDiversification_GitHubCopy/3.DataSplit')

## 1B. Canonicalize the reaction and add to the pandas dataframe
Canonicalize the reaction SMILES to help remove duplicates.

In [4]:
#load the necessary modules
from rdkit.Chem import rdChemReactions
import rdkit.Chem as Chem

RDKit WARNING: [10:07:06] Enabling RDKit 2019.09.3 jupyter extensions


In [5]:
#initial definitions
debug = False
count = 0
canonical_rxn_smiles_list = []

for row in dataset.itertuples():
    
    count += 1
    
    if debug and count == 5:
        break
    

    if debug:
        print ('The original reaction is: {}'.format (row[4]))
    
    #split reactants and product
    all_reactants, product = row[4].split('>>')
    
    #remove atom mapping information from the product 
    product_mol = Chem.MolFromSmiles(product)
    [a.ClearProp('molAtomMapNumber') for a in product_mol.GetAtoms()]
    prod_smi = Chem.MolToSmiles(product_mol, True)
    
    #remove atom mapping information from the reactant
    reactants = [Chem.MolFromSmiles(smi) for smi in all_reactants.split('.')]
    reactants_smi_list = []
    for react in reactants:
        [a.ClearProp('molAtomMapNumber') for a in react.GetAtoms()]
        reactants_smi_list.append(Chem.MolToSmiles(react, True))
    reactants_smi = '.'.join(reactants_smi_list)

    if debug:
        print ('Ammended reaction is {}>>{}'.format (reactants_smi,prod_smi))
    
    #load the reaction into RDKit and have it translate into SMILES
    rxn = rdChemReactions.ReactionFromSmarts('{}>>{}'.format(reactants_smi,prod_smi),useSmiles = True)
    rxn_smiles_canonical = rdChemReactions.ReactionToSmiles(rxn)

    if debug:
        print ('Canonical reaction smiles is: {}'.format (rxn_smiles_canonical))
        print ('-------------------------------')

    canonical_rxn_smiles_list.append (rxn_smiles_canonical)

In [6]:
dataset ['canonical_rxn_smiles'] = canonical_rxn_smiles_list

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 477394 entries, 0 to 479034
Data columns (total 7 columns):
Reaction SMILES                       477394 non-null object
reactionID                            477394 non-null int64
react_smiles                          477394 non-null object
rxn_smiles                            477394 non-null object
template                              470654 non-null object
Template Verification (True/False)    477394 non-null bool
canonical_rxn_smiles                  477394 non-null object
dtypes: bool(1), int64(1), object(5)
memory usage: 26.0+ MB


## 1C. How many duplicates are there?

In [8]:
# Understand the duplicates
from collections import Counter
pre_removal = Counter (dataset['canonical_rxn_smiles'])

In [9]:
Counter(pre_removal.values())

Counter({1: 425244,
         4: 770,
         3: 2574,
         2: 18490,
         5: 324,
         6: 150,
         7: 81,
         8: 44,
         26: 1,
         9: 28,
         10: 21,
         19: 1,
         16: 2,
         11: 9,
         33: 1,
         12: 6,
         25: 1,
         14: 2,
         18: 2,
         13: 2,
         21: 1,
         15: 2,
         20: 1})

In [10]:
pre_removal

Counter({'CON(C)C(=O)c1ccc(O)nc1>>CC(C)CC(=O)c1ccc(O)nc1': 1,
         'O=C(O)c1ccc(Cl)c([N+](=O)[O-])c1>>CNc1ccc(C(=O)O)cc1[N+](=O)[O-]': 1,
         'CCn1cc(C(=O)O)c(=O)c2cc(F)c(-c3ccc(N)cc3)cc21>>CCn1cc(C(=O)O)c(=O)c2cc(F)c(-c3ccc(NC=O)cc3)cc21': 1,
         'COCC(C)Oc1cc(Oc2cnc(C(=O)N3CCC3)cn2)cc(C(=O)O)c1>>COCC(C)Oc1cc(Oc2cnc(C(=O)N3CCC3)cn2)cc(C(=O)Nc2cnc(C)cn2)c1': 4,
         'Clc1cc2c(Cl)nc(-c3ccncc3)nc2s1>>Clc1cc2c(NCc3ccc(Cl)c(Cl)c3)nc(-c3ccncc3)nc2s1': 1,
         'Cc1c(Cl)nnc(C(C#N)c2ccc(F)c(C#N)c2)c1C>>Cc1c(Cl)nnc(Cc2ccc(F)c(C#N)c2)c1C': 1,
         'O=C(N1CCc2ccc(Cl)c(OS(=O)(=O)C(F)(F)F)c2CC1)C(F)(F)F>>CC(Nc1c(Cl)ccc2c1CCN(C(=O)C(F)(F)F)CC2)c1ccc(F)c(Cl)c1': 1,
         'CC(C)N1CCN(C(=O)c2ccc3[nH]c(C(=O)N4CCN(S(C)(=O)=O)CC4)cc3c2)CC1>>CCOC(=O)N1CCN(C(=O)c2cc3cc(C(=O)N4CCN(C(C)C)CC4)ccc3[nH]2)CC1': 1,
         'CN1C(=O)C(N)c2ccccc2-c2ccccc21>>CC(C(=O)NCc1ccc(F)cc1)C(=O)NC1C(=O)N(C)c2ccccc2-c2ccccc21': 1,
         'CC(=O)N1CCN(c2ccc(N)nc2)CC1>>CC(=O)N1CCN(c2ccc(NC(=O)Cc3cc

In [11]:
pre_removal['CON(C)C(=O)c1ccc(O)nc1>>CC(C)CC(=O)c1ccc(O)nc1']

1

There are enough duplicates to worry about this problem. Remove duplicate reactions, only keeping the first instance!

## 1D. Remove duplicate transformations

In [12]:
dataset.head(2)

,Reaction SMILES,reactionID,react_smiles,rxn_smiles,template,Template Verification (True/False),canonical_rxn_smiles
0,[CH2:15]([CH:16]([CH3:17])[CH3:18])[Mg+:19].[C...,0,CON(C)C(=O)c1ccc(O)nc1,CON(C)[C:6]([c:5]1[cH:4][n:3][c:2]([OH:1])[cH:...,[#7;a:5]:[c:4]:[c:3]-[C;H0;D3;+0:1](=[O;D1;H0:...,True,CON(C)C(=O)c1ccc(O)nc1>>CC(C)CC(=O)c1ccc(O)nc1
1,[CH3:14][NH2:15].[N+:1](=[O:2])([O-:3])[c:4]1[...,1,O=C(O)c1ccc(Cl)c([N+](=O)[O-])c1,Cl[c:12]1[c:4]([N+:1](=[O:2])[O-:3])[cH:5][c:6...,[Cl;H0;D1;+0]-[c;H0;D3;+0:1](:[c:2]):[c:3]>>C-...,True,O=C(O)c1ccc(Cl)c([N+](=O)[O-])c1>>CNc1ccc(C(=O...


In [13]:
#growing list of reactions to include
rxns_included = []

#keep track of lists
Reaction_SMILES = []
reactionID = []
react_smiles = []
rxn_smiles = []
template = []
Template_Verification_TF = []
canonical_rxn_smiles = []

#debug settings
count = 0
debug = False

for row in dataset.itertuples():
    
    #increment count
    count += 1
    
    #print the value of count
    if count % 5000 == 0:
        print ('On count:{}'.format (count))
    
    #if debug, then break at 5000
    if debug and count == 5000:
        break
    
    #if canonicalized SMILES occurs only once, then save it to the list structure
    if pre_removal [row[7]] == 1:
        Reaction_SMILES.append (row [1])
        reactionID.append (row[2])
        react_smiles.append (row[3])
        rxn_smiles.append (row[4])
        template.append (row[5])
        Template_Verification_TF.append (row[6])
        canonical_rxn_smiles.append(row[7])
        
    #if canonicalized SMILES occurs more than once, then check for duplicates and save only
    #one instance
    if pre_removal[row[7]] != 1:
        
        # if the reaction is already there, do not include it
        if row[7] in set (rxns_included):
            if debug:
                print ('Reaction excluded: {}'.format (row[7]))
            continue
        #if it is not there, include it in the database
        Reaction_SMILES.append (row [1])
        reactionID.append (row[2])
        react_smiles.append (row[3])
        rxn_smiles.append (row[4])
        template.append (row[5])
        Template_Verification_TF.append (row[6])
        canonical_rxn_smiles.append(row[7])
        #add to the reaction included list
        rxns_included.append (row[7])

On count:5000
On count:10000
On count:15000
On count:20000
On count:25000
On count:30000
On count:35000
On count:40000
On count:45000
On count:50000
On count:55000
On count:60000
On count:65000
On count:70000
On count:75000
On count:80000
On count:85000
On count:90000
On count:95000
On count:100000
On count:105000
On count:110000
On count:115000
On count:120000
On count:125000
On count:130000
On count:135000
On count:140000
On count:145000
On count:150000
On count:155000
On count:160000
On count:165000
On count:170000
On count:175000
On count:180000
On count:185000
On count:190000
On count:195000
On count:200000
On count:205000
On count:210000
On count:215000
On count:220000
On count:225000
On count:230000
On count:235000
On count:240000
On count:245000
On count:250000
On count:255000
On count:260000
On count:265000
On count:270000
On count:275000
On count:280000
On count:285000
On count:290000
On count:295000
On count:300000
On count:305000
On count:310000
On count:315000
On count:320

In [14]:
dataset_v2 = pd.DataFrame({'Reaction SMILES': Reaction_SMILES,
                          'reactionID': reactionID,
                          'react_smiles': react_smiles,
                           'rxn_smiles': rxn_smiles,
                          'template': template,
                          'Template Verification (True/False)': Template_Verification_TF,
                          'canonical_rxn_smiles': canonical_rxn_smiles})

In [15]:
dataset_v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447757 entries, 0 to 447756
Data columns (total 7 columns):
Reaction SMILES                       447757 non-null object
reactionID                            447757 non-null int64
react_smiles                          447757 non-null object
rxn_smiles                            447757 non-null object
template                              441341 non-null object
Template Verification (True/False)    447757 non-null bool
canonical_rxn_smiles                  447757 non-null object
dtypes: bool(1), int64(1), object(5)
memory usage: 20.9+ MB


## 1E. Double check to make sure

In [16]:
post_removal = Counter (dataset_v2 ['canonical_rxn_smiles'])

In [17]:
Counter (post_removal.values())

Counter({1: 447757})

# 2. Split by compound rather than reaction

## 2a. Remove Template Verification = False

In [18]:
Counter (dataset_v2['Template Verification (True/False)'])

Counter({True: 435246, False: 12511})

In [19]:
dataset_v2 = dataset_v2 [dataset_v2['Template Verification (True/False)'] == True]

In [20]:
Counter (dataset_v2['Template Verification (True/False)'])

Counter({True: 435246})

## 2b. Understand the number of duplicate compounds

In [21]:
compound_duplicates = Counter (dataset_v2['react_smiles'])

In [22]:
Counter (compound_duplicates.values())

Counter({1: 231523,
         6: 1696,
         2: 26401,
         7: 1178,
         40: 8,
         9: 647,
         31: 24,
         15: 184,
         3: 9156,
         12: 332,
         5: 2595,
         4: 4579,
         18: 116,
         8: 887,
         99: 1,
         23: 59,
         39: 16,
         43: 12,
         74: 3,
         77: 2,
         19: 105,
         10: 491,
         53: 5,
         25: 48,
         20: 101,
         75: 1,
         76: 2,
         30: 35,
         29: 35,
         16: 161,
         11: 348,
         13: 261,
         80: 2,
         63: 2,
         36: 17,
         123: 2,
         45: 6,
         72: 3,
         56: 4,
         14: 226,
         21: 68,
         42: 12,
         24: 50,
         46: 5,
         48: 7,
         22: 69,
         49: 5,
         35: 13,
         57: 6,
         17: 118,
         55: 5,
         190: 1,
         33: 20,
         73: 2,
         26: 43,
         69: 2,
         105: 1,
         28: 26,
         47:

## 2b. Split by compound
Split by compound + Keep the number of reactions in train:val:test (0.8:0.1:0.1)

In [25]:
import numpy as np

#dictionary for splitting strategy
split_strategy = {}

import time

#split the dataset
def split_data (cmpd_duplicates_dict, val_frac = 0.1, test_frac = 0.1, shuffle = True, seed = 123):
    
    debug = False
    
    # Define shuffling
    if shuffle:
        if seed is None:
            np.random.seed(int(time.time()))
        else:
            np.random.seed(seed)
        def shuffle_func(x):
            np.random.shuffle(x)
    else:
        def shuffle_func(x):
            pass
    
    #get the list of chemical compounds
    compound_list = list (cmpd_duplicates_dict.keys())
    
    if debug:
        print ('The original compound list is: {}'.format (compound_list))
    
    #shuffle the list
    shuffle_func (compound_list)
    
    if debug:
        print ('The shuffled compound list is: {}'.format (compound_list))
    
    N = sum (list(cmpd_duplicates_dict.values()))
    
    train_end = int((1.0 - val_frac - test_frac) * N)
    val_end = int((1.0 - test_frac) * N)
    
    compound_count = 0
    
    #for quality control

    train_count = 0
    val_count = 0
    test_count = 0
    
    for item in compound_list:
        
        if compound_count < train_end:
            split_strategy[item] = 'train'
            compound_count += compound_duplicates [item]
            train_count += compound_duplicates [item]
            if debug:
                print ('Train compound: {}'.format (item))
            
        elif compound_count < val_end:
            split_strategy[item] = 'val'
            compound_count += compound_duplicates [item]
            val_count += compound_duplicates [item]
            if debug:
                print ('Validation compound: {}'.format (item))
            
        else:
            split_strategy[item] = 'test'
            compound_count += compound_duplicates [item]
            test_count += compound_duplicates [item]
            if debug:
                print ('Test compound: {}'.format (item))
    

    print ('{} train compounds'. format (train_count))
    print ('{} validation compounds'. format (val_count))
    print ('{} test compounds'. format (test_count))
    print ('{} total compounds'.format (compound_count))

In [26]:
split_data (compound_duplicates)

348196 train compounds
43525 validation compounds
43525 test compounds
435246 total compounds


## 2C. Add split information to Pandas dataframe

In [27]:
split_strategy

{'COC(=O)c1cc(Nc2nc(N3CCCCC3)nc3ccccc23)[nH]n1': 'train',
 'CC(=O)c1ccc(Cl)c([N+](=O)[O-])c1': 'train',
 'COC(=O)c1cc(C)c2[nH]c(C)nc2n1': 'train',
 'Fc1ccc2c(c1)CCO2': 'train',
 'CC(C)c1ccccc1S(=O)(=O)Cl': 'train',
 'FC(F)(F)c1cc(Br)c2cc[nH]c2c1': 'train',
 'CC(=O)SCC(Cc1ccccc1)C(=O)NC(C)(C)C(=O)OCc1ccccc1': 'train',
 'COC(=O)c1ccc(C#CCNS(C)(=O)=O)s1': 'train',
 'O=c1[nH]cnc2ncccc12': 'train',
 'O=S(=O)(Oc1ccc(OCC2CCCCO2)nc1Cc1ccccc1)C(F)(F)F': 'train',
 'CC(C)CN(C(CO)CCCCNC(=O)OCC1c2ccccc2-c2ccccc21)S(=O)(=O)c1ccc(N)cc1': 'train',
 'CCc1cc(CN2CC(C(=O)OC)C2)sc1-c1noc(-c2ccc(Oc3cccc(F)c3)c(F)c2)n1': 'train',
 'CCOC(=O)C1CCCN(c2ccc(-c3ccncc3)cc2F)C1': 'train',
 'COc1ccc(-c2ccc(C=O)s2)cc1': 'train',
 'CCOC(=O)c1cc2ccc(OC3CCN(C(C)C)CC3)cc2s1': 'train',
 'COC(=O)CN(Cc1ccc(OC)cc1)S(=O)(=O)c1ccc(OCCCCl)cc1': 'train',
 'COc1cccc2c1Oc1cc(Cl)ccc1N2C1CCN(C(=O)OC(C)(C)C)CC1': 'train',
 'O=C(O)C(O)Cc1ccccc1': 'train',
 'C=C(C)c1cccc(Br)n1': 'train',
 'O=Cc1c(O)cc(O)cc1F': 'train',
 'C#CCn1ccnc1': '

In [28]:
split_strategy['CCOC(=O)c1ccc(N(C)C)cc1']

'train'

In [29]:
dataset_v2.head()

,Reaction SMILES,reactionID,react_smiles,rxn_smiles,template,Template Verification (True/False),canonical_rxn_smiles
0,[CH2:15]([CH:16]([CH3:17])[CH3:18])[Mg+:19].[C...,0,CON(C)C(=O)c1ccc(O)nc1,CON(C)[C:6]([c:5]1[cH:4][n:3][c:2]([OH:1])[cH:...,[#7;a:5]:[c:4]:[c:3]-[C;H0;D3;+0:1](=[O;D1;H0:...,True,CON(C)C(=O)c1ccc(O)nc1>>CC(C)CC(=O)c1ccc(O)nc1
1,[CH3:14][NH2:15].[N+:1](=[O:2])([O-:3])[c:4]1[...,1,O=C(O)c1ccc(Cl)c([N+](=O)[O-])c1,Cl[c:12]1[c:4]([N+:1](=[O:2])[O-:3])[cH:5][c:6...,[Cl;H0;D1;+0]-[c;H0;D3;+0:1](:[c:2]):[c:3]>>C-...,True,O=C(O)c1ccc(Cl)c([N+](=O)[O-])c1>>CNc1ccc(C(=O...
2,[CH2:1]([CH3:2])[n:3]1[cH:4][c:5]([C:22](=[O:2...,2,CCn1cc(C(=O)O)c(=O)c2cc(F)c(-c3ccc(N)cc3)cc21,[CH2:1]([CH3:2])[n:3]1[cH:4][c:5]([C:22](=[O:2...,[NH2;D1;+0:1]-[c:2]>>O=C-[NH;D2;+0:1]-[c:2],True,CCn1cc(C(=O)O)c(=O)c2cc(F)c(-c3ccc(N)cc3)cc21>...
3,[Cl:1][C:2]([N:3]([CH3:4])[CH3:5])=[C:6]([CH3:...,3,COCC(C)Oc1cc(Oc2cnc(C(=O)N3CCC3)cn2)cc(C(=O)O)c1,O=[C:25]([c:24]1[cH:23][c:22]([O:21][c:18]2[cH...,[O;H0;D1;+0]=[C;H0;D3;+0:1](-[OH;D1;+0:2])-[c:...,True,COCC(C)Oc1cc(Oc2cnc(C(=O)N3CCC3)cn2)cc(C(=O)O)...
4,[Cl:11][c:12]1[c:13]2[c:14]([n:15][c:16](-[c:1...,4,Clc1cc2c(Cl)nc(-c3ccncc3)nc2s1,Cl[c:12]1[c:13]2[c:14]([n:15][c:16](-[c:18]3[c...,[#16;a:6]:[c:5]1:[#7;a:4]:[c:3]:[#7;a:2]:[c;H0...,True,Clc1cc2c(Cl)nc(-c3ccncc3)nc2s1>>Clc1cc2c(NCc3c...


In [30]:
dataset = []
for row in dataset_v2.itertuples():
    dataset.append (split_strategy[row[3]])

In [31]:
Counter (dataset)

Counter({'train': 348196, 'val': 43525, 'test': 43525})

In [32]:
dataset_v2 ['dataset'] = dataset

In [34]:
dataset_v2.sample(10)

,Reaction SMILES,reactionID,react_smiles,rxn_smiles,template,Template Verification (True/False),canonical_rxn_smiles,dataset
420405,[Br:19][CH2:20][c:21]1[cH:22][cH:23][cH:24][cH...,448299,O=c1nc(-c2ccccn2)sc2ccc(O)cc12,[OH:1][c:2]1[cH:3][cH:4][c:5]2[c:6]([c:7](=[O:...,[OH;D1;+0:2]-[c:1]>>[c:1]-[O;H0;D2;+0:2]-C-c1:...,True,O=c1nc(-c2ccccn2)sc2ccc(O)cc12>>O=c1nc(-c2cccc...,train
392991,[Br:10][CH2:11][CH2:12][CH3:13].[C:1]([CH3:2])...,417713,CC(=O)C1=CNCCC1,[C:1]([CH3:2])(=[O:3])[C:4]1=[CH:5][NH:6][CH2:...,[C:4]=[C:3]-[NH;D2;+0:1]-[C:2]>>C-C-C-[N;H0;D3...,True,CC(=O)C1=CNCCC1>>CCCN1C=C(C(C)=O)CCC1,val
72246,[Br:20][N:21]1[C:22](=[O:23])[CH2:24][CH2:25][...,73422,CS(=O)(=O)c1ccc(C(=CCC2CCCC2)C(=O)O)cc1,O=[C:36]([C:35](=[CH:34][CH2:33][CH:28]1[CH2:2...,[C:1]=[C:2]-[C;H0;D3;+0:3](=[O;H0;D1;+0])-[OH;...,True,CS(=O)(=O)c1ccc(C(=CCC2CCCC2)C(=O)O)cc1>>CS(=O...,test
85119,[CH3:22][S-:23].[CH3:30][CH2:31][O:32][C:33]([...,86689,COc1cc(N2CCC(CCI)CC2)c(C)cc1[N+](=O)[O-],I[CH2:2][CH2:3][CH:4]1[CH2:5][CH2:6][N:7]([c:1...,[C:2]-[CH2;D2;+0:1]-[I;H0;D1;+0]>>C-S-[CH2;D2;...,True,COc1cc(N2CCC(CCI)CC2)c(C)cc1[N+](=O)[O-]>>COc1...,train
413239,[CH2:1]([c:2]1[cH:3][cH:4][cH:5][cH:6][cH:7]1)...,440280,c1ccc(COc2ccc(-c3nnc(C4(c5cccs5)CCC4)n3C3CC3)c...,c1ccc(C[O:8][c:9]2[cH:10][cH:11][c:12](-[c:15]...,[c:2]-[O;H0;D2;+0:1]-[C;H2;D2;+0]-[c;H0;D3;+0]...,True,c1ccc(COc2ccc(-c3nnc(C4(c5cccs5)CCC4)n3C3CC3)c...,train
10691,[C:39]([O:40][BH-:41]([O:42][C:43](=[O:44])[CH...,10747,NC1CCC(C(=O)Nc2c(C(=O)Nc3ccc(Cl)cn3)oc3ccccc23...,[NH2:3][CH:4]1[CH2:5][CH2:6][CH:7]([C:10](=[O:...,[C:2]-[NH2;D1;+0:1]>>O=C1-C-C-C-[N;H0;D3;+0:1]...,True,NC1CCC(C(=O)Nc2c(C(=O)Nc3ccc(Cl)cn3)oc3ccccc23...,train
341085,[C:1]([OH:2])(=[O:3])[CH:4]([CH:5]([C:6]([OH:7...,360281,CCOC(=O)COc1cc(C2CCCNC2)ccc1C,[CH2:11]([CH3:12])[O:13][C:14]([CH2:15][O:16][...,[C:2]-[NH;D2;+0:1]-[C:3]>>F-C(-F)(-F)-c1:c:c:c...,True,CCOC(=O)COc1cc(C2CCCNC2)ccc1C>>CCOC(=O)COc1cc(...,train
319728,[CH3:1][CH:2]1[N:3]([CH2:7][CH2:8][CH2:9][O:10...,336753,CC(=O)Nc1cnn(-c2ccc(OCCCN3CCCC3C)cc2)c1,[CH3:1][CH:2]1[N:3]([CH2:7][CH2:8][CH2:9][O:10...,[C;D1;H3:3]-[C:2](=[O;D1;H0:4])-[NH;D2;+0:1]-[...,True,CC(=O)Nc1cnn(-c2ccc(OCCCN3CCCC3C)cc2)c1>>CC(=O...,train
321429,[CH3:1][O:2][CH2:3][c:4]1[cH:5][cH:6][c:7]([O:...,338602,COCc1ccc(Oc2cc(OC3CCOCC3)c3[nH]c(C(N)=O)cc3c2)cn1,O=[C:27]([c:15]1[cH:14][c:13]2[cH:12][c:11]([O...,[#7;a:4]:[c:3]-[C;H0;D3;+0:1](-[N;D1;H2:2])=[O...,True,COCc1ccc(Oc2cc(OC3CCOCC3)c3[nH]c(C(N)=O)cc3c2)...,train
88576,[C:1]1([c:7]2[cH:8][c:9]3[c:10]([n:11][cH:12]2...,90259,Cc1nc(-c2cn(S(=O)(=O)c3ccccc3)c3ncc(C4=CCCCC4)...,[C:1]1([c:7]2[cH:8][c:9]3[c:10]([n:11][cH:12]2...,[#7;a:1]:[c:2]:[c:3]-[C;H0;D3;+0:4](-[C:5])=[C...,True,Cc1nc(-c2cn(S(=O)(=O)c3ccccc3)c3ncc(C4=CCCCC4)...,train


# 3. Compute reaction partner (s)

In [35]:
import re
import rdkit.Chem.AllChem as AllChem
import rdkit.Chem as Chem

In [36]:
from rdkit.Chem.Draw import rdMolDraw2D

from IPython.display import SVG

from rdkit.Chem import rdChemReactions

def draw_rxn(rxn, unmap=True, width=800, height=200):
    if unmap:
        for mol in rxn.GetReactants():
            [a.SetAtomMapNum(0) for a in mol.GetAtoms()]
        for mol in rxn.GetProducts():
            [a.SetAtomMapNum(0) for a in mol.GetAtoms()]
    drawer = rdMolDraw2D.MolDraw2DSVG(width, height)
    drawer.DrawReaction(rxn)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText().replace('svg:', '')
    display(SVG(svg))

def draw_rxn_smi(rxn_smi, unmap=True, width=800, height=200):
    rxn = AllChem.ReactionFromSmarts(rxn_smi, useSmiles=True)
    draw_rxn(rxn, unmap=unmap, width=width, height=height)

In [37]:
#list saves the new reaction partners
rxn_partners_new = []

#debug settings
debug = False

#if debug, set up a counter
if debug:
    count = 0

#cumulative react_partners
react_partners_all = []
    
#loop through the dataset
for row in dataset_v2.itertuples():
    
    #exit strategy for debug
    if debug:
        count += 1
        if count == 5:
            break
    
    #provide a running counter
    if row[0] % 5000 == 0:
        print ('On index {:d}'.format (int (row[0])))
    
    #draw out the reaction SMILES
    if debug:
        print ('*************New Reaction*****************')
        print ('The raw reaction:')
        draw_rxn_smi (row[1])
        print ('----------------')
        print ('The principal reactant is: {}'.format (row[3]))
    
    #get reactants and products
    all_reactants, all_products = row[1].split('>>')
    
    #get atom map numbers of all product molecules
    
    #parse the products
    products = [Chem.MolFromSmiles(smi) for smi in all_products.split('.')]
    
    #setup prod_maps dictionary
    prod_maps = set()
    
    #loop through the product 
    for prod in products:
        
        #convert the prod mol to smiles
        prod_smi = Chem.MolToSmiles (prod, True)
        
        if debug:
            print ('The prod_smi considered is: {}'. format (prod_smi))
            print ('The relevant atom map numbers is: {}'. format (set (re.findall('\:([[0-9]+)\]', prod_smi))))
        #add map numbers to growing prod_maps set
        prod_maps_temp = set (re.findall('\:([[0-9]+)\]', prod_smi))
        prod_maps = prod_maps.union(prod_maps_temp)
    
    #if debug, print prod_maps
    if debug:
        print ('The product maps are: {}'.format (prod_maps))
    
    #get reactant smiles that correspond to the product maps, and are not the principal reactant
    react_partners = []
    
    #parse reactants and convert them to mols
    reactants = [Chem.MolFromSmiles(smi) for smi in all_reactants.split('.')]
    
    #multiple reactants = enumerate
    for react in reactants:
        
        #Avoid if reactants don't have atom mapping numbers
        if not any([a.HasProp('molAtomMapNumber') for a in react.GetAtoms()]):
            continue
        
        #check if reactants contribute to products
        used = False
        
        #loop through atoms in the reactant to see if it was used
        for a in react.GetAtoms():
            
            #if reactant atom has atom map number
            if a.HasProp('molAtomMapNumber'):
                
                #if that atom map number is part of the product maps
                if a.GetProp('molAtomMapNumber') in prod_maps:
                    used = True
        
        #if it is used, clear atom map numbers, check if it is principal reactant
        if used:
            #clear atom map numbers
            [a.ClearProp('molAtomMapNumber') for a in react.GetAtoms()]
            if debug:
                print ('This reactant was used: {}'.format (Chem.MolToSmiles(react, True)))
            #check if it is principal reactant
            if Chem.MolToSmiles(react, True) != row[3]:
                react_partners.append (Chem.MolToSmiles(react, True))
    if debug:
        print ('The corresponding reaction partners are: {}'.format (react_partners))
    
    #append to reaction partners
    react_partners_all.append (react_partners)

if debug:
    print ('**********Final Result************')
    print ('The react_partners_all list: {}'.format (react_partners_all))

On index 0


/home/karthiksankar2/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: Possible nested set at position 4
RDKit WARNING: [10:18:57] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:18:57] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:18:57] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:18:57] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:18:57] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:18:57] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:18:57] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:18:57] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:18:57] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:18:57] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:18:57] WARNING: not removing 

RDKit WARNING: [10:19:02] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:02] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:02] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:02] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:02] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:02] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:02] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:02] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:02] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:02] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:02] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:02] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:02] WARNING: not r

RDKit WARNING: [10:19:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:06] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:06] WARNING: not r

RDKit WARNING: [10:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:19:11] WARNING: not r

...

RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors


On index 445000


RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:54] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:54] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:54] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:54] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:54] WARNING: not r

RDKit WARNING: [10:43:58] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:58] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:58] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:58] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:58] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:58] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:58] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:58] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:58] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:58] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:58] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:58] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:58] WARNING: not r

RDKit WARNING: [10:44:03] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:44:03] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:44:03] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:44:03] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:44:03] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:44:03] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:44:03] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:44:03] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:44:03] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:44:03] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:44:03] WARNING: not removing hydrogen atom without neighbors


In [38]:
len (react_partners_all)

435246

In [39]:
dataset_v2['str_Reaction_Partner'] = react_partners_all

In [40]:
dataset_v2.head(1)

,Reaction SMILES,reactionID,react_smiles,rxn_smiles,template,Template Verification (True/False),canonical_rxn_smiles,dataset,str_Reaction_Partner
0,[CH2:15]([CH:16]([CH3:17])[CH3:18])[Mg+:19].[C...,0,CON(C)C(=O)c1ccc(O)nc1,CON(C)[C:6]([c:5]1[cH:4][n:3][c:2]([OH:1])[cH:...,[#7;a:5]:[c:4]:[c:3]-[C;H0;D3;+0:1](=[O;D1;H0:...,True,CON(C)C(=O)c1ccc(O)nc1>>CC(C)CC(=O)c1ccc(O)nc1,train,[CC(C)C[Mg+]]


# 4. Check the dataset

In [41]:
dataset_v2.head(1)

,Reaction SMILES,reactionID,react_smiles,rxn_smiles,template,Template Verification (True/False),canonical_rxn_smiles,dataset,str_Reaction_Partner
0,[CH2:15]([CH:16]([CH3:17])[CH3:18])[Mg+:19].[C...,0,CON(C)C(=O)c1ccc(O)nc1,CON(C)[C:6]([c:5]1[cH:4][n:3][c:2]([OH:1])[cH:...,[#7;a:5]:[c:4]:[c:3]-[C;H0;D3;+0:1](=[O;D1;H0:...,True,CON(C)C(=O)c1ccc(O)nc1>>CC(C)CC(=O)c1ccc(O)nc1,train,[CC(C)C[Mg+]]


In [42]:
dataset_v2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 435246 entries, 0 to 447756
Data columns (total 9 columns):
Reaction SMILES                       435246 non-null object
reactionID                            435246 non-null int64
react_smiles                          435246 non-null object
rxn_smiles                            435246 non-null object
template                              435246 non-null object
Template Verification (True/False)    435246 non-null bool
canonical_rxn_smiles                  435246 non-null object
dataset                               435246 non-null object
str_Reaction_Partner                  435246 non-null object
dtypes: bool(1), int64(1), object(7)
memory usage: 30.3+ MB


## 4a. Are the canonicalized reaction SMILES unique?

In [43]:
from collections import Counter

In [44]:
Counter (Counter (dataset_v2['canonical_rxn_smiles']).values())

Counter({1: 435246})

The analysis above suggests that the canonicalized reaction SMILES are indeed unique!

## 4b. Is there compound overlap between training, validation, and test sets?

In [45]:
train = dataset_v2 [dataset_v2['dataset']=='train']
val = dataset_v2 [dataset_v2['dataset']=='val']
test = dataset_v2 [dataset_v2['dataset']=='test']

In [46]:
train.head(1)

,Reaction SMILES,reactionID,react_smiles,rxn_smiles,template,Template Verification (True/False),canonical_rxn_smiles,dataset,str_Reaction_Partner
0,[CH2:15]([CH:16]([CH3:17])[CH3:18])[Mg+:19].[C...,0,CON(C)C(=O)c1ccc(O)nc1,CON(C)[C:6]([c:5]1[cH:4][n:3][c:2]([OH:1])[cH:...,[#7;a:5]:[c:4]:[c:3]-[C;H0;D3;+0:1](=[O;D1;H0:...,True,CON(C)C(=O)c1ccc(O)nc1>>CC(C)CC(=O)c1ccc(O)nc1,train,[CC(C)C[Mg+]]


In [47]:
train_compound = set (train['react_smiles'].tolist())
val_compound = set (val['react_smiles'].tolist())
test_compound = set (test['react_smiles'].tolist())

In [48]:
len (train_compound.intersection (train_compound))

225146

In [49]:
len (train_compound.intersection (val_compound))

0

In [50]:
len (train_compound.intersection (test_compound))

0

In [51]:
len (val_compound.intersection (train_compound))

0

In [52]:
len (val_compound.intersection (val_compound))

28526

In [53]:
len (val_compound.intersection (test_compound))

0

In [54]:
len (test_compound.intersection (train_compound))

0

In [55]:
len (test_compound.intersection (val_compound))

0

In [56]:
len (test_compound.intersection (test_compound))

28248

## 4c. Are you only considering cases where you can extract and apply templates?

In [58]:
dataset_v2.head(1)

,Reaction SMILES,reactionID,react_smiles,rxn_smiles,template,Template Verification (True/False),canonical_rxn_smiles,dataset,str_Reaction_Partner
0,[CH2:15]([CH:16]([CH3:17])[CH3:18])[Mg+:19].[C...,0,CON(C)C(=O)c1ccc(O)nc1,CON(C)[C:6]([c:5]1[cH:4][n:3][c:2]([OH:1])[cH:...,[#7;a:5]:[c:4]:[c:3]-[C;H0;D3;+0:1](=[O;D1;H0:...,True,CON(C)C(=O)c1ccc(O)nc1>>CC(C)CC(=O)c1ccc(O)nc1,train,[CC(C)C[Mg+]]


In [59]:
Counter (dataset_v2['Template Verification (True/False)'])

Counter({True: 435246})

## 4d. Are the 'str_Reaction_Partner' and 'canonical_rxn_smiles' extractions good?

In [61]:
dataset_v2_5 = dataset_v2.sample (5)

In [64]:
for row in dataset_v2_5.itertuples():
    print ('*'*100)
    print ('Reaction SMILES: {}'.format (row[1]))
    print ('Principal reactant: {}'. format (row[3]))
    print ('Atom mapped reaction smiles: {}'.format (row[4]))
    print ('Canonical reaction smiles: {}'.format (row[7]))
    print ('str_Reaction_Partner: {}'.format (row[9]))

****************************************************************************************************
Reaction SMILES: [CH3:38][c:39]1[cH:40][cH:41][cH:42][cH:43][cH:44]1.[CH3:8][C:9]1([CH3:31])[N:10]=[C:11]([c:25]2[cH:26][cH:27][cH:28][cH:29][cH:30]2)[c:12]2[c:13]3[c:14]([c:15]([NH2:19])[cH:16][c:17]2[CH2:18]1)[O:20][C:21]([CH3:23])([CH3:24])[CH2:22]3.[Na+:37].[Na:32][O:33][C:34]#[N:35].[OH-:36].[OH:1][C:2]([C:3]([F:4])([F:5])[F:6])=[O:7]>>[CH3:8][C:9]1([CH3:31])[N:10]=[C:11]([c:25]2[cH:26][cH:27][cH:28][cH:29][cH:30]2)[c:12]2[c:13]3[c:14]([c:15]([NH:19][C:34](=[O:33])[NH2:35])[cH:16][c:17]2[CH2:18]1)[O:20][C:21]([CH3:23])([CH3:24])[CH2:22]3
Principal reactant: CC1(C)Cc2cc(N)c3c(c2C(c2ccccc2)=N1)CC(C)(C)O3
Atom mapped reaction smiles: [CH3:8][C:9]1([CH3:31])[N:10]=[C:11]([c:25]2[cH:26][cH:27][cH:28][cH:29][cH:30]2)[c:12]2[c:13]3[c:14]([c:15]([NH2:19])[cH:16][c:17]2[CH2:18]1)[O:20][C:21]([CH3:23])([CH3:24])[CH2:22]3>>NC(=O)[NH:19][c:15]1[c:14]2[c:13]([c:12]3[c:17]([cH:16]1)[CH2:18][C:9]

## 4e. Save the files

In [65]:
dataset_v2.to_csv('dataset_input.csv')

In [72]:
dataset_v2_check = pd.read_csv('dataset_input.csv', index_col = 0)

In [68]:
dataset_v2.head(1)

,Reaction SMILES,reactionID,react_smiles,rxn_smiles,template,Template Verification (True/False),canonical_rxn_smiles,dataset,str_Reaction_Partner
0,[CH2:15]([CH:16]([CH3:17])[CH3:18])[Mg+:19].[C...,0,CON(C)C(=O)c1ccc(O)nc1,CON(C)[C:6]([c:5]1[cH:4][n:3][c:2]([OH:1])[cH:...,[#7;a:5]:[c:4]:[c:3]-[C;H0;D3;+0:1](=[O;D1;H0:...,True,CON(C)C(=O)c1ccc(O)nc1>>CC(C)CC(=O)c1ccc(O)nc1,train,[CC(C)C[Mg+]]


In [69]:
dataset_v2_check.head(1)

,Reaction SMILES,reactionID,react_smiles,rxn_smiles,template,Template Verification (True/False),canonical_rxn_smiles,dataset,str_Reaction_Partner
0,[CH2:15]([CH:16]([CH3:17])[CH3:18])[Mg+:19].[C...,0,CON(C)C(=O)c1ccc(O)nc1,CON(C)[C:6]([c:5]1[cH:4][n:3][c:2]([OH:1])[cH:...,[#7;a:5]:[c:4]:[c:3]-[C;H0;D3;+0:1](=[O;D1;H0:...,True,CON(C)C(=O)c1ccc(O)nc1>>CC(C)CC(=O)c1ccc(O)nc1,train,['CC(C)C[Mg+]']


In [70]:
from ast import literal_eval
dataset_v2_check['str_Reaction_Partner'] =  dataset_v2_check.str_Reaction_Partner.apply(lambda x: literal_eval(str(x)))

In [73]:
dataset_v2.equals (dataset_v2_check)

False